### DA380A - Seminar 1
Group 3 - Ahmed Saber Elsayed Radwan, Lakshmi Vishal Hayagrivan, Sam El Saati

### Importing Libraries
In this cell, we import the necessary Python libraries for data manipulation, machine learning, and evaluation.

In [138]:
import re
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

### Reading Data
This cell reads data from two csv files, the concatenates them and displays a random head.

In [70]:
# Define the file paths
file_path1 = r"business.csv"
file_path2 = r"economy.csv"

# Read the CSV files with the correct separator
data1 = pd.read_csv(file_path1, sep=',', header=0)  # Business data
data2 = pd.read_csv(file_path2, sep=',', header=0)  # Economy data

# Add a new column to indicate the source of the data
data1['class'] = 'business'  # Add 'business' label to business data
data2['class'] = 'economy'    # Add 'economy' label to economy data

# Concatenate the two datasets
combined_data = pd.concat([data1, data2], ignore_index=True)

# Shuffle the combined dataset
shuffled_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the first few rows of the combined dataset
shuffled_data.head(20)


,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,09-03-2022,AirAsia,I5,423,14:55,Mumbai,07h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:25,Bangalore,"2,074",economy
1,21-02-2022,Air India,AI,469,05:15,Delhi,17h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Raipur\n\t...,23:05,Mumbai,"10,260",economy
2,11-03-2022,Air India,AI,415,16:45,Delhi,24h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:35,Kolkata,"32,347",business
3,03-03-2022,Air India,AI,840,20:50,Hyderabad,25h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:20,Kolkata,"51,707",business
4,08-03-2022,Vistara,UK,875,19:20,Mumbai,20h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,15:40,Delhi,"41,269",business
5,24-03-2022,Vistara,UK,774,20:35,Kolkata,11h 15m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,07:50,Bangalore,"6,195",economy
6,10-03-2022,Indigo,6E,6491,13:05,Bangalore,09h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,Kolkata,"4,392",economy
7,27-03-2022,Vistara,UK,738,18:45,Kolkata,25h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:50,Hyderabad,"5,760",economy
8,11-03-2022,Indigo,6E,2479,10:25,Delhi,08h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:20,Chennai,"4,821",economy
9,25-02-2022,Vistara,UK,963,08:50,Delhi,12h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"53,937",business


### Checking for Missing Values
This cell checks for missing values in all columns.

In [72]:
missing_data_per_column = shuffled_data.isnull().sum()
print(missing_data_per_column)

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
class         0
dtype: int64


### Converting Price to Numeric
This cell converts 'price' to numeric values.

In [74]:
# Step 1: Convert 'price' to numeric
# shuffled_data['price'] = shuffled_data['price'].astype(str).str.replace(r'[^0-9.,]', '', regex=True)  # Remove non-numeric characters
shuffled_data['price'] = shuffled_data['price'].str.replace(r'[^0-9]', '', regex=True)
shuffled_data['price'] = pd.to_numeric(shuffled_data['price'], errors='coerce')
# Display the first few rows of the combined dataset
shuffled_data.head(20)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,09-03-2022,AirAsia,I5,423,14:55,Mumbai,07h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:25,Bangalore,2074,economy
1,21-02-2022,Air India,AI,469,05:15,Delhi,17h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Raipur\n\t...,23:05,Mumbai,10260,economy
2,11-03-2022,Air India,AI,415,16:45,Delhi,24h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:35,Kolkata,32347,business
3,03-03-2022,Air India,AI,840,20:50,Hyderabad,25h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:20,Kolkata,51707,business
4,08-03-2022,Vistara,UK,875,19:20,Mumbai,20h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,15:40,Delhi,41269,business
5,24-03-2022,Vistara,UK,774,20:35,Kolkata,11h 15m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,07:50,Bangalore,6195,economy
6,10-03-2022,Indigo,6E,6491,13:05,Bangalore,09h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,Kolkata,4392,economy
7,27-03-2022,Vistara,UK,738,18:45,Kolkata,25h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:50,Hyderabad,5760,economy
8,11-03-2022,Indigo,6E,2479,10:25,Delhi,08h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:20,Chennai,4821,economy
9,25-02-2022,Vistara,UK,963,08:50,Delhi,12h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,53937,business


### Displaying Data
This cell displays our data.

In [76]:
shuffled_data.head(20)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,09-03-2022,AirAsia,I5,423,14:55,Mumbai,07h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:25,Bangalore,2074,economy
1,21-02-2022,Air India,AI,469,05:15,Delhi,17h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Raipur\n\t...,23:05,Mumbai,10260,economy
2,11-03-2022,Air India,AI,415,16:45,Delhi,24h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:35,Kolkata,32347,business
3,03-03-2022,Air India,AI,840,20:50,Hyderabad,25h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:20,Kolkata,51707,business
4,08-03-2022,Vistara,UK,875,19:20,Mumbai,20h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,15:40,Delhi,41269,business
5,24-03-2022,Vistara,UK,774,20:35,Kolkata,11h 15m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,07:50,Bangalore,6195,economy
6,10-03-2022,Indigo,6E,6491,13:05,Bangalore,09h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,Kolkata,4392,economy
7,27-03-2022,Vistara,UK,738,18:45,Kolkata,25h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:50,Hyderabad,5760,economy
8,11-03-2022,Indigo,6E,2479,10:25,Delhi,08h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:20,Chennai,4821,economy
9,25-02-2022,Vistara,UK,963,08:50,Delhi,12h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,53937,business


### Dropping Columns
In this cell, we drop the 'ch_code', 'num_code' columns from our dataset.

In [78]:
# Drop unnecessary columns
shuffled_data = shuffled_data.drop(columns=['ch_code', 'num_code'], errors='ignore')
shuffled_data.head(20)

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,class
0,09-03-2022,AirAsia,14:55,Mumbai,07h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:25,Bangalore,2074,economy
1,21-02-2022,Air India,05:15,Delhi,17h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Raipur\n\t...,23:05,Mumbai,10260,economy
2,11-03-2022,Air India,16:45,Delhi,24h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:35,Kolkata,32347,business
3,03-03-2022,Air India,20:50,Hyderabad,25h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:20,Kolkata,51707,business
4,08-03-2022,Vistara,19:20,Mumbai,20h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,15:40,Delhi,41269,business
5,24-03-2022,Vistara,20:35,Kolkata,11h 15m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,07:50,Bangalore,6195,economy
6,10-03-2022,Indigo,13:05,Bangalore,09h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,Kolkata,4392,economy
7,27-03-2022,Vistara,18:45,Kolkata,25h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:50,Hyderabad,5760,economy
8,11-03-2022,Indigo,10:25,Delhi,08h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:20,Chennai,4821,economy
9,25-02-2022,Vistara,08:50,Delhi,12h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,53937,business


### Encoding Stops & Dropping Unnecessary Columns
This cell defines a function that encodes the 'stop' column, converting categories like 'non-stop' or '1-stop' into numeric values (e.g., 0 for non-stop, 1 for 1-stop, etc.). The original 'stop' column is then dropped.

In [80]:
# Drop unnecessary columns
# shuffled_data = shuffled_data.drop(columns=['stop'], errors='ignore')
# shuffled_data.head(20)

# Function to encode 'stop' values
def encode_stops(stop_value):
    if 'non-stop' in stop_value:
        return 0
    elif '1-stop' in stop_value:
        return 1
    elif '2' in stop_value:  # This handles both '2-stop' and '2+ stops'
        return 2
    else:
        return -1  # In case there's any unexpected value, we return -1 as a fallback

# Apply the function to the 'stop' column and store the results in a new column
shuffled_data['stop_encoded'] = shuffled_data['stop'].apply(encode_stops)

# Drop the original 'stop' column, as it's now encoded
shuffled_data = shuffled_data.drop(columns=['stop'], errors='ignore')

# Display the first few rows of the updated DataFrame
shuffled_data.head(20)

,date,airline,dep_time,from,time_taken,arr_time,to,price,class,stop_encoded
0,09-03-2022,AirAsia,14:55,Mumbai,07h 30m,22:25,Bangalore,2074,economy,1
1,21-02-2022,Air India,05:15,Delhi,17h 50m,23:05,Mumbai,10260,economy,1
2,11-03-2022,Air India,16:45,Delhi,24h 50m,17:35,Kolkata,32347,business,1
3,03-03-2022,Air India,20:50,Hyderabad,25h 30m,22:20,Kolkata,51707,business,1
4,08-03-2022,Vistara,19:20,Mumbai,20h 20m,15:40,Delhi,41269,business,1
5,24-03-2022,Vistara,20:35,Kolkata,11h 15m,07:50,Bangalore,6195,economy,1
6,10-03-2022,Indigo,13:05,Bangalore,09h 35m,22:40,Kolkata,4392,economy,1
7,27-03-2022,Vistara,18:45,Kolkata,25h 05m,19:50,Hyderabad,5760,economy,1
8,11-03-2022,Indigo,10:25,Delhi,08h 55m,19:20,Chennai,4821,economy,1
9,25-02-2022,Vistara,08:50,Delhi,12h 05m,20:55,Hyderabad,53937,business,1


### Converting date format & finding min, max
This pyhton code in this cell converts 'date' to datetime, ensuring the format is correct, then calculates and displays min and max flight date.

In [82]:
# # Convert 'date' to datetime
# shuffled_data['date'] = pd.to_datetime(shuffled_data['date'], format='%d-%m-%Y', errors='coerce')

# shuffled_data.head(20)
# Convert 'date' to datetime, ensuring the format is correct
shuffled_data['date'] = pd.to_datetime(shuffled_data['date'], format='%d-%m-%Y', errors='coerce')

# Get the minimum and maximum dates from the 'date' column
min_date = shuffled_data['date'].min()
max_date = shuffled_data['date'].max()

# Display the minimum and maximum dates
print(f"Minimum flight date: {min_date}")
print(f"Maximum flight date: {max_date}")


Minimum flight date: 2022-02-11 00:00:00
Maximum flight date: 2022-03-31 00:00:00


### Calculating Days Left Until Flight
In this cell, we calculate the number of days left until each flight, relative to a reference date (2022-02-10). The 'date' column is dropped after calculating the days left.

In [84]:
# Define the reference date as 2022-02-10
reference_date = pd.Timestamp('2022-02-10')

# Calculate the days left for each flight
shuffled_data['days_left'] = (shuffled_data['date'] - reference_date).dt.days

shuffled_data = shuffled_data.drop(columns=['date'])  # Drop the 'date' column

# Display the first few rows of the updated DataFrame
shuffled_data.head()


,airline,dep_time,from,time_taken,arr_time,to,price,class,stop_encoded,days_left
0,AirAsia,14:55,Mumbai,07h 30m,22:25,Bangalore,2074,economy,1,27
1,Air India,05:15,Delhi,17h 50m,23:05,Mumbai,10260,economy,1,11
2,Air India,16:45,Delhi,24h 50m,17:35,Kolkata,32347,business,1,29
3,Air India,20:50,Hyderabad,25h 30m,22:20,Kolkata,51707,business,1,21
4,Vistara,19:20,Mumbai,20h 20m,15:40,Delhi,41269,business,1,26


### Converting time into right format
This cell converts departure and arrival times to the right format.

In [86]:

# Convert 'dep_time' and 'arr_time' to appropriate formats
shuffled_data['dep_time'] = pd.to_datetime(shuffled_data['dep_time'], format='%H:%M', errors='coerce')
shuffled_data['arr_time'] = pd.to_datetime(shuffled_data['arr_time'], format='%H:%M', errors='coerce')

shuffled_data.head(20)

,airline,dep_time,from,time_taken,arr_time,to,price,class,stop_encoded,days_left
0,AirAsia,1900-01-01 14:55:00,Mumbai,07h 30m,1900-01-01 22:25:00,Bangalore,2074,economy,1,27
1,Air India,1900-01-01 05:15:00,Delhi,17h 50m,1900-01-01 23:05:00,Mumbai,10260,economy,1,11
2,Air India,1900-01-01 16:45:00,Delhi,24h 50m,1900-01-01 17:35:00,Kolkata,32347,business,1,29
3,Air India,1900-01-01 20:50:00,Hyderabad,25h 30m,1900-01-01 22:20:00,Kolkata,51707,business,1,21
4,Vistara,1900-01-01 19:20:00,Mumbai,20h 20m,1900-01-01 15:40:00,Delhi,41269,business,1,26
5,Vistara,1900-01-01 20:35:00,Kolkata,11h 15m,1900-01-01 07:50:00,Bangalore,6195,economy,1,42
6,Indigo,1900-01-01 13:05:00,Bangalore,09h 35m,1900-01-01 22:40:00,Kolkata,4392,economy,1,28
7,Vistara,1900-01-01 18:45:00,Kolkata,25h 05m,1900-01-01 19:50:00,Hyderabad,5760,economy,1,45
8,Indigo,1900-01-01 10:25:00,Delhi,08h 55m,1900-01-01 19:20:00,Chennai,4821,economy,1,29
9,Vistara,1900-01-01 08:50:00,Delhi,12h 05m,1900-01-01 20:55:00,Hyderabad,53937,business,1,15


### Binning 'dep_time' and 'arr_time'
The code in this cell bins departure and arrival times into categories.

In [88]:
# Function to categorize times into different bins
def categorize_time(time):
    if pd.isnull(time):
        return 'Unknown'  # Handle missing or invalid times
    hour = time.hour  # Extract the hour directly from the Timestamp object
    if 4 <= hour < 7:
        return 'Early Morning'
    elif 7 <= hour < 11:
        return 'Morning'
    elif 11 <= hour < 15:
        return 'Noon'
    elif 15 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 23:
        return 'Evening'
    else:
        return 'Late Night'

# Apply the function to 'dep_time' and 'arr_time' columns
shuffled_data['dep_time_category'] = shuffled_data['dep_time'].apply(categorize_time)
shuffled_data['arr_time_category'] = shuffled_data['arr_time'].apply(categorize_time)

# Display the first few rows of the updated DataFrame
shuffled_data[['dep_time', 'dep_time_category', 'arr_time', 'arr_time_category']].head()

shuffled_data.head(20)

,airline,dep_time,from,time_taken,arr_time,to,price,class,stop_encoded,days_left,dep_time_category,arr_time_category
0,AirAsia,1900-01-01 14:55:00,Mumbai,07h 30m,1900-01-01 22:25:00,Bangalore,2074,economy,1,27,Noon,Evening
1,Air India,1900-01-01 05:15:00,Delhi,17h 50m,1900-01-01 23:05:00,Mumbai,10260,economy,1,11,Early Morning,Late Night
2,Air India,1900-01-01 16:45:00,Delhi,24h 50m,1900-01-01 17:35:00,Kolkata,32347,business,1,29,Afternoon,Afternoon
3,Air India,1900-01-01 20:50:00,Hyderabad,25h 30m,1900-01-01 22:20:00,Kolkata,51707,business,1,21,Evening,Evening
4,Vistara,1900-01-01 19:20:00,Mumbai,20h 20m,1900-01-01 15:40:00,Delhi,41269,business,1,26,Evening,Afternoon
5,Vistara,1900-01-01 20:35:00,Kolkata,11h 15m,1900-01-01 07:50:00,Bangalore,6195,economy,1,42,Evening,Morning
6,Indigo,1900-01-01 13:05:00,Bangalore,09h 35m,1900-01-01 22:40:00,Kolkata,4392,economy,1,28,Noon,Evening
7,Vistara,1900-01-01 18:45:00,Kolkata,25h 05m,1900-01-01 19:50:00,Hyderabad,5760,economy,1,45,Evening,Evening
8,Indigo,1900-01-01 10:25:00,Delhi,08h 55m,1900-01-01 19:20:00,Chennai,4821,economy,1,29,Morning,Evening
9,Vistara,1900-01-01 08:50:00,Delhi,12h 05m,1900-01-01 20:55:00,Hyderabad,53937,business,1,15,Morning,Evening


### Converting 'time_taken' to total hours
In this cell, we convert 'time_taken' to total hours.

In [90]:
# Function to convert 'time_taken' to total hours
def convert_time_taken(time_str):
    # Find all occurrences of hours and minutes
    match = re.match(r'(?:(\d+)h)?\s*(?:(\d+)m)?', time_str.strip())
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        # Convert total time to hours
        total_hours = round(hours + minutes / 60, 2)
        return total_hours
    return 0

# Apply the function to the 'time_taken' column
shuffled_data['time_taken_in_hours'] = shuffled_data['time_taken'].apply(convert_time_taken)

shuffled_data = shuffled_data.drop(columns=['time_taken'])  # Drop the 'time_taken' column
shuffled_data.head(10)

,airline,dep_time,from,arr_time,to,price,class,stop_encoded,days_left,dep_time_category,arr_time_category,time_taken_in_hours
0,AirAsia,1900-01-01 14:55:00,Mumbai,1900-01-01 22:25:00,Bangalore,2074,economy,1,27,Noon,Evening,7.50
1,Air India,1900-01-01 05:15:00,Delhi,1900-01-01 23:05:00,Mumbai,10260,economy,1,11,Early Morning,Late Night,17.83
2,Air India,1900-01-01 16:45:00,Delhi,1900-01-01 17:35:00,Kolkata,32347,business,1,29,Afternoon,Afternoon,24.83
3,Air India,1900-01-01 20:50:00,Hyderabad,1900-01-01 22:20:00,Kolkata,51707,business,1,21,Evening,Evening,25.50
4,Vistara,1900-01-01 19:20:00,Mumbai,1900-01-01 15:40:00,Delhi,41269,business,1,26,Evening,Afternoon,20.33
5,Vistara,1900-01-01 20:35:00,Kolkata,1900-01-01 07:50:00,Bangalore,6195,economy,1,42,Evening,Morning,11.25
6,Indigo,1900-01-01 13:05:00,Bangalore,1900-01-01 22:40:00,Kolkata,4392,economy,1,28,Noon,Evening,9.58
7,Vistara,1900-01-01 18:45:00,Kolkata,1900-01-01 19:50:00,Hyderabad,5760,economy,1,45,Evening,Evening,25.08
8,Indigo,1900-01-01 10:25:00,Delhi,1900-01-01 19:20:00,Chennai,4821,economy,1,29,Morning,Evening,8.92
9,Vistara,1900-01-01 08:50:00,Delhi,1900-01-01 20:55:00,Hyderabad,53937,business,1,15,Morning,Evening,12.08


### Dropping 'arr_time', 'dep_time' columns
This cell drops the original 'arr_time', 'dep_time' columns, as they are now encoded.

In [92]:
# Drop the original 'arr_time', 'dep_time' columns
shuffled_data = shuffled_data.drop(columns=['arr_time', 'dep_time'], errors='ignore')
shuffled_data.head(20)

,airline,from,to,price,class,stop_encoded,days_left,dep_time_category,arr_time_category,time_taken_in_hours
0,AirAsia,Mumbai,Bangalore,2074,economy,1,27,Noon,Evening,7.50
1,Air India,Delhi,Mumbai,10260,economy,1,11,Early Morning,Late Night,17.83
2,Air India,Delhi,Kolkata,32347,business,1,29,Afternoon,Afternoon,24.83
3,Air India,Hyderabad,Kolkata,51707,business,1,21,Evening,Evening,25.50
4,Vistara,Mumbai,Delhi,41269,business,1,26,Evening,Afternoon,20.33
5,Vistara,Kolkata,Bangalore,6195,economy,1,42,Evening,Morning,11.25
6,Indigo,Bangalore,Kolkata,4392,economy,1,28,Noon,Evening,9.58
7,Vistara,Kolkata,Hyderabad,5760,economy,1,45,Evening,Evening,25.08
8,Indigo,Delhi,Chennai,4821,economy,1,29,Morning,Evening,8.92
9,Vistara,Delhi,Hyderabad,53937,business,1,15,Morning,Evening,12.08


### Encoding categorical columns
In this step, we use one hot encoder to encode categorical columns into numeric data.

In [114]:
# Define the updated categorical columns including dep_time_category and arr_time_category
categorical_cols = ['airline', 'from', 'to', 'class', 'dep_time_category', 'arr_time_category']

# Define non-categorical columns in the correct order (manually specify the correct order)
non_categorical_cols = ['price',  'stop_encoded', 'days_left', 'time_taken_in_hours']  # Explicit order

# Create a ColumnTransformer to apply OneHotEncoder to the categorical columns
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Keep the other columns unchanged
)

# Fit and transform the data
shuffled_data_encoded = column_transformer.fit_transform(shuffled_data)

# Convert the sparse matrix to a dense array
shuffled_data_encoded_dense = shuffled_data_encoded.toarray()

# Get feature names for the one-hot encoded columns
onehot_feature_names = column_transformer.named_transformers_['onehot'].get_feature_names_out(categorical_cols)

# Ensure the order of non-categorical columns is correct by explicitly defining it
# Combine the one-hot encoded feature names with the other columns that were not encoded
final_columns = list(onehot_feature_names) + non_categorical_cols  # Proper order

# Create a new DataFrame with the encoded features and specified column order
encoded_df = pd.DataFrame(shuffled_data_encoded_dense, columns=final_columns)

# Display the first few rows of the encoded DataFrame
encoded_df.head(20)

,airline_Air India,airline_AirAsia,airline_GO FIRST,airline_Indigo,airline_SpiceJet,airline_StarAir,airline_Trujet,airline_Vistara,from_Bangalore,from_Chennai,...,arr_time_category_Afternoon,arr_time_category_Early Morning,arr_time_category_Evening,arr_time_category_Late Night,arr_time_category_Morning,arr_time_category_Noon,price,stop_encoded,days_left,time_taken_in_hours
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2074.0,1.0,27.0,7.50
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,10260.0,1.0,11.0,17.83
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,32347.0,1.0,29.0,24.83
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,51707.0,1.0,21.0,25.50
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,41269.0,1.0,26.0,20.33
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,6195.0,1.0,42.0,11.25
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,4392.0,1.0,28.0,9.58
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,5760.0,1.0,45.0,25.08
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,4821.0,1.0,29.0,8.92
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,53937.0,1.0,15.0,12.08


### Exploratory Analysis
This cell displays data to be able to take a look at it.

In [122]:
encoded_df.describe()

,airline_Air India,airline_AirAsia,airline_GO FIRST,airline_Indigo,airline_SpiceJet,airline_StarAir,airline_Trujet,airline_Vistara,from_Bangalore,from_Chennai,...,arr_time_category_Afternoon,arr_time_category_Early Morning,arr_time_category_Evening,arr_time_category_Late Night,arr_time_category_Morning,arr_time_category_Noon,price,stop_encoded,days_left,time_taken_in_hours
count,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,...,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000,300261.000000
mean,0.269412,0.053613,0.077190,0.143608,0.030011,0.000203,0.000137,0.425826,0.173536,0.128888,...,0.128635,0.008969,0.386137,0.125870,0.213354,0.137034,20883.717666,0.924213,26.004473,12.217780
std,0.443655,0.225253,0.266892,0.350693,0.170617,0.014252,0.011685,0.494468,0.378710,0.335076,...,0.334796,0.094279,0.486864,0.331704,0.409677,0.343884,22695.911266,0.398188,13.560985,7.192984
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1105.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4783.000000,1.000000,15.000000,6.750000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7425.000000,1.000000,26.000000,11.250000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,42521.000000,1.000000,38.000000,16.170000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,123071.000000,2.000000,49.000000,49.830000


### Splitting Data and Training the Machine Learning Models
In the following code, we split data into training and testing data. We train various machine learning models using the training data to learn patterns in the dataset, then we test them to evaluate. After that Features are displayed arranged by importance.

In [140]:

# Define the features (X) and target (y)
X = encoded_df.drop(columns=['price'])  # Features (drop the 'price' column)
y = encoded_df['price']  # Target column is 'price'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80% train, 20% test

# Function to train and evaluate a model
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    # Print results with formatted outputs
    print(f"\n{model_name} - Performance:")
    print(f"R²: {r2:.2f}")
    print(f"MAPE: {mape:.3f}")
    
    # Get feature importances if supported
    if hasattr(model, 'feature_importances_'):
        feature_importances = model.feature_importances_
        feature_importance_df = pd.DataFrame({
            'Feature': X.columns,
            'Importance': feature_importances
        }).sort_values(by='Importance', ascending=False)
        print(f"\n{model_name} - Top 10 Feature Importances:\n", feature_importance_df.head(10))
    
    return round(r2, 2), round(mape, 3)

# Random Forest
rf_model = RandomForestRegressor(random_state=42)
r2_rf, mape_rf = evaluate_model(rf_model, X_train, X_test, y_train, y_test, "Random Forest")

# Gradient Boosting
gb_model = GradientBoostingRegressor(random_state=42)
r2_gb, mape_gb = evaluate_model(gb_model, X_train, X_test, y_train, y_test, "Gradient Boosting")

# Decision Tree Regression
dt_model = DecisionTreeRegressor(random_state=42)
r2_dt, mape_dt = evaluate_model(dt_model, X_train, X_test, y_train, y_test, "Decision Tree Regression")

# Now you can compare them later on
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting', 'Decision Tree Regression'],
    'R²': [r2_rf, r2_gb, r2_dt],
    'MAPE': [mape_rf, mape_gb, mape_dt]
})

print("\nModel Comparison:\n", comparison_df)


Random Forest - Performance:
R²: 0.99
MAPE: 0.069

Random Forest - Top 10 Feature Importances:
                 Feature  Importance
21        class_economy    0.501482
20       class_business    0.378302
36  time_taken_in_hours    0.057764
35            days_left    0.018565
7       airline_Vistara    0.005520
0     airline_Air India    0.004377
10           from_Delhi    0.003541
16             to_Delhi    0.003190
13          from_Mumbai    0.002145
19            to_Mumbai    0.002032

Gradient Boosting - Performance:
R²: 0.95
MAPE: 0.207

Gradient Boosting - Top 10 Feature Importances:
                 Feature  Importance
20       class_business    0.546177
21        class_economy    0.379678
36  time_taken_in_hours    0.043068
35            days_left    0.011553
7       airline_Vistara    0.007300
0     airline_Air India    0.003472
34         stop_encoded    0.001311
12         from_Kolkata    0.001272
18           to_Kolkata    0.001255
17         to_Hyderabad    0.000864

Decis

### Evaluation Results

In [ ]:
Rank1: Random Forest  R² = 0.99  MAPE = 0.069

In [ ]:
Rank2: Decision Tree Regression R² = 0.98  MAPE = 0.074

In [ ]:
Rank3: Gradient Boosting R² = 0.95  MAPE = 0.207

In [ ]:
Measuring R² and MAPE both show that the Random Forest model is the best fit model for our dataset.